In [16]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import openpolicedata as opd

In [17]:
# Query for the entire table of available data
datasets = opd.datasets_query()
datasets.head()

,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,date_field,dataset_id,jurisdiction_field
0,Arizona,Gilbert,Gilbert,STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,date,<NA>,<NA>
1,Arizona,Mesa,Mesa,STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,date,<NA>,<NA>
2,Arizona,State Patrol,State Patrol,STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,date,<NA>,<NA>
3,Arkansas,Little Rock,Little Rock,TRAFFIC STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,date,<NA>,<NA>
4,California,Anaheim,Anaheim,TRAFFIC STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,date,<NA>,<NA>


In [18]:
# Query for all available datasets from Maryland
df = opd.datasets_query(state="Maryland")
df.head()

,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,date_field,dataset_id,jurisdiction_field
37,Maryland,Baltimore,Baltimore,ARRESTS,MULTI,Arrests in the City of Baltimore,ArcGIS,https://egis.baltimorecity.gov/egis/rest/servi...,ArrestDateTime,<NA>,<NA>
38,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2017,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,calldatetime,<NA>,<NA>
39,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2018,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,calldatetime,<NA>,<NA>
40,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2019,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,calldatetime,<NA>,<NA>
41,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2020,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,calldatetime,<NA>,<NA>


In [19]:
# Query for all stops data containing both traffic and investigatory stops in Virginia
df = opd.datasets_query(table_type=opd.TableTypes.STOPS, state="Virginia")
df.head()

,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,date_field,dataset_id,jurisdiction_field
104,Virginia,Virginia Community Policing Act,MULTI,STOPS,MULTI,A data collection consisting of all traffic an...,Socrata,data.virginia.gov,incident_date,2c96-texw,agency_name


In [20]:
# To access the data, create a source using a Source Name (usually a police department name). There is an optional state input to clarify ambiguities.
# Create data source for the Virginia Community Policing Act

src = opd.Source(source_name="Virginia Community Policing Act", state="Virginia")
src.datasets.head()

,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,date_field,dataset_id,jurisdiction_field
104,Virginia,Virginia Community Policing Act,MULTI,STOPS,MULTI,A data collection consisting of all traffic an...,Socrata,data.virginia.gov,incident_date,2c96-texw,agency_name


In [21]:
# Find out what types of data are available from this source

types = src.get_tables_types()

print(types)

['STOPS']


In [22]:
# Find out what years are available from the stops table

years = src.get_years(table_type=types[0])
print(years)

[2020, 2021, 2022]


In [23]:
# Find out what jurisdictions are contained in the stops table
jurisdictions = src.get_jurisdictions()
print(jurisdictions)

["Accomack County Sheriff's Office", 'Thomas Nelson Comm. College Police', 'Carilion Health System Police Dept.', 'Salem Police Department', 'Virginia State Police', 'Gretna Police Department', "Petersburg City Sheriff's Office", 'Portsmouth Police Department', "Loudoun County Sheriff's Office", 'Remington Police Department', "Dickenson County Sheriff's Office", 'Gate City Police Department', "Middlesex County Sheriff's Office", 'South Boston Police Department', 'Farmville Police Department', 'Bedford Police Department', 'Tappahannock Police Department', 'Fauquier County Fire Marshals Office', "Powhatan Sheriff's Office", 'Lebanon Police Department', 'Norfolk Airport Authority Police Department', 'Falls Church Police Department', 'Germanna Community College Police Department', 'Front Royal Police Department', 'VMI Police Department', 'Va. Lottery Dept.', 'Clifton Forge Police Dept', 'St. Paul Police Department', 'Berryville Police Department', 'Wytheville Police Department', 'Charlotte

In [24]:
# Find out what jurisdictions are contained in the stops table that containt the word "Arlington"
jurisdictions = src.get_jurisdictions(partial_name="Arlington")
print(jurisdictions)

["Arlington County Sheriff's Office", 'Arlington County Police Department']


In [25]:
# Load data from the source for the Arlington County Police Department in 2021
jurisdiction = "Arlington County Police Department"
t = src.load_from_url(year=2021, table_type=opd.TableTypes.STOPS, jurisdiction_filter=jurisdiction)

In [26]:
# Show the first 5 rows of the table
t.table.head(n=5)

,incident_date,agency_name,jurisdiction,reason_for_stop,race,ethnicity,age,gender,action_taken,violation_type,specific_violation,person_searched,vehicle_searched,additional_arrest
0,2021-01-01,Arlington County Police Department,ARLINGTON CO,OTHER,WHITE,HISPANIC,44,FEMALE,WARNING,LOCAL,14.2-16,NO,NO,NO
1,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,23,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
2,2021-01-01,Arlington County Police Department,ARLINGTON CO,TRAFFIC VIOLATION,BLACK OR AFRICAN AMERICAN,NON-HISPANIC,27,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
3,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,CITATION/SUMMONS,COMMONWEALTH,46.2-301,NO,NO,NO
4,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,WARNING,COMMONWEALTH,46.2-612,NO,NO,NO


In [27]:
from pathlib import Path
# Save to CSV file with default filename. Need double forward slashes in Windows.
output_dir = Path('.') / 'Data'

if not output_dir.exists():
    os.makedirs(output_dir)

if output_dir.is_dir():
    t.to_csv(outputDir=output_dir)
    # For custom filename uncomment below
    # t.to_csv(outputDir=output_dir, filename="custom_filename.csv")     
    print(f"Created csv files in {output_dir.resolve()}")
else:
    print(f"Unable to create the directory {output_dir.resolve()}")

Created csv files in /home/user/cjc/openpolicedata/notebooks/Data


In [28]:
# Load the CSV file back in

# Do not really need to create a new source. Could use t instead
new_src = opd.Source(source_name="Virginia Community Policing Act")
new_t = new_src.load_from_csv(outputDir=output_dir, year=2021, jurisdiction_filter=jurisdiction)

In [29]:
new_t.table.head()

,incident_date,agency_name,jurisdiction,reason_for_stop,race,ethnicity,age,gender,action_taken,violation_type,specific_violation,person_searched,vehicle_searched,additional_arrest
0,2021-01-01,Arlington County Police Department,ARLINGTON CO,OTHER,WHITE,HISPANIC,44,FEMALE,WARNING,LOCAL,14.2-16,NO,NO,NO
1,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,23,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
2,2021-01-01,Arlington County Police Department,ARLINGTON CO,TRAFFIC VIOLATION,BLACK OR AFRICAN AMERICAN,NON-HISPANIC,27,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
3,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,CITATION/SUMMONS,COMMONWEALTH,46.2-301,NO,NO,NO
4,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,WARNING,COMMONWEALTH,46.2-612,NO,NO,NO
